In [ ]:
Install all required libraries.
(Skip this cell if you already have these installed in your environment.)
!pip install pennylane pennylane-qiskit qiskit qiskit-aer qiskit-ibm-runtime qiskit-algorithms

Pennylane to Qiskit Translation Examples

This notebook contains **five examples** of translating common **PennyLane** patterns
into their **Qiskit** equivalents:

1. Single-qubit gate (`qml.RX`) -> `RXGate`
2. Shot-based simulation (`default.qubit` vs `AerSimulator`)
3. Noise model workflow (`NoiseModel` in PennyLane vs Aer `NoiseModel` in Qiskit)
4. Running on IBM Quantum hardware (PennyLane `qiskit.remote` vs Qiskit Runtime `EstimatorV2`)
5. Variational circuit + optimization (`GradientDescentOptimizer` -> `GradientDescent`)

The goal is to:
- show *structural* equivalence between the frameworks,
- highlight where semantics differ (e.g. observables, qubit ordering, noise insertion),
- and give commented reference snippets you can adapt in real projects.

In [1]:
import numpy as np
from functools import partial

#1. Gate Translation: `qml.RX` → `qiskit.circuit.library.RXGate`

**What we’re doing:**
- In PennyLane: build a circuit with a single `RX(θ)` and a measurement, *without* executing it.
- In Qiskit: build a `QuantumCircuit` with `RXGate(θ)` and measurement, *without* executing it.

This is a **pure circuit-construction** example (no simulator / backend).

In [ ]:
import pennylane as qml

theta = 0.3

# Use a QuantumTape to *record* operations, not execute them.
tape = qml.tape.QuantumTape()
with tape:
    qml.RX(theta, wires=0)
    qml.measure(0)

# Print the PennyLane circuit structure
print(tape.draw())

0: ──RX──┤↗├─┤  



##Qiskit equivalent


In [39]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(1)
theta = 0.3
qc.rx(theta, 0)
qc.measure_all()

# Print the Qiskit circuit structure
print(qc)


        ┌─────────┐ ░ ┌─┐
     q: ┤ Rx(0.3) ├─░─┤M├
        └─────────┘ ░ └╥┘
meas: 1/═══════════════╩═
                       0 


## 2. Simulator Backend

**What we’re doing:**
- Create a 3-qubit GHZ-like circuit.
- Run with a finite number of shots.
- Collect classical bitstring counts.

### Mapping

| PennyLane              | Qiskit                          |
|------------------------|---------------------------------|
| Device                 | `default.qubit` (shots set)    |
| Measurement            | `qml.counts`                   |
| Simulator backend      | `AerSimulator`                 |
| Result                 | dict of bitstring → count      |


In [41]:
import pennylane as qml

dev = qml.device("default.qubit", wires=3, shots=2000)

@qml.qnode(dev)
def ghz_counts():
    """Create a 3-qubit GHZ-like state and return shot counts."""
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    return qml.counts(wires=[0, 1, 2])

print("[PennyLane] counts:", ghz_counts())

[PennyLane] counts: {np.str_('000'): np.int64(1006), np.str_('111'): np.int64(994)}


In [42]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.measure_all()

sim = AerSimulator(shots=2000)
result = sim.run(qc).result()
counts = result.get_counts()

print("[Qiskit]    counts:", counts)

[Qiskit]    counts: {'111': 991, '000': 1009}


## 3. Noise Model Workflow

**What we’re doing:**
- Define a noise model that targets **all 2-qubit gates**.
- In PennyLane: use `NoiseModel` + `BooleanFn` + `add_noise` to insert noise channels.
- In Qiskit: use Aer `NoiseModel` and attach depolarizing errors to all 2-qubit gate names.

### Design choice

In both frameworks, we conceptually say:
> "Whenever a 2-qubit gate appears, apply depolarizing noise to those qubits."

In [ ]:
import pennylane as qml
from pennylane import numpy as np

prob_depol = 0.05

1. Condition: "is this operation a 2-qubit gate?"
@qml.BooleanFn
def is_two_qubit_gate(op, **metadata):
    """Return True if the operation 'op' acts on exactly 2 wires."""
    return len(op.wires) == 2

# 2. Noise: apply single-qubit DepolarizingChannel on *each* wire of that gate
def two_qubit_depol(op, **metadata):
    """For any 2-qubit gate 'op', apply a depolarizing channel on each of its wires."""
    for w in op.wires:
        qml.DepolarizingChannel(prob_depol, wires=w)

# 3. Build the noise model: mapping from BooleanFn -> noise function
noise_model = qml.NoiseModel({is_two_qubit_gate: two_qubit_depol})

# 4. Base mixed-state device (supports channels)
base_dev = qml.device("default.mixed", wires=2, shots=1000)

# 5. Wrap device with noise model (device-level add_noise is recommended for channels)
noisy_dev = qml.add_noise(base_dev, noise_model)

# 6. Example circuit with a 2-qubit gate
@qml.qnode(noisy_dev)
def noisy_cnot_counts():
    """A simple circuit where CNOT triggers depolarizing noise on both qubits."""
    qml.Hadamard(0)        # unaffected by this particular noise model
    qml.CNOT(wires=[0, 1]) # <- noise will be inserted after this
    return qml.counts()

print("[PennyLane] Noisy counts:", noisy_cnot_counts())

[PennyLane] Noisy counts: {np.str_('00'): np.int64(470), np.str_('01'): np.int64(26), np.str_('10'): np.int64(42), np.str_('11'): np.int64(462)}


In [46]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, errors

# -------------------------------
# 1. Build noise model
# -------------------------------

noise_model = NoiseModel()

# List of common 2-qubit gate names to target with noise.
# You can extend this list as needed for your backend.
two_qubit_gates = [
    "cx", "cz", "swap",
    "iswap", "ecr", "rxx", "ryy", "rzz", "rzx",
]

# 2-qubit depolarizing error (acts jointly on the 2-qubit subspace)
twoq_error = errors.depolarizing_error(prob_depol, 2)

# Attach this error to *all* 2-qubit gates on *all* qubits
noise_model.add_all_qubit_quantum_error(twoq_error, two_qubit_gates)

# -------------------------------
# 2. Example circuit
# -------------------------------

qc = QuantumCircuit(2)
qc.h(0)             # 1-qubit gate, NOT noised by our model
qc.cx(0, 1)         # 2-qubit gate, WILL have depolarizing error applied
qc.measure_all()

# -------------------------------
# 3. Simulate with noisy backend
# -------------------------------

sim = AerSimulator(noise_model=noise_model, shots=1000)
result = sim.run(qc).result()

print("[Qiskit]    Noisy counts:", result.get_counts())

[Qiskit]    Noisy counts: {'01': 17, '10': 11, '00': 476, '11': 496}


## 4. Running on Real Hardware (IBM Quantum)

**What we’re doing:**
- PennyLane: create a `qiskit.remote` device backed by an IBM Quantum backend via `QiskitRuntimeService`, then define a QNode and call it.
- Qiskit: build a logical circuit, compile to ISA (backend native gates), then use `EstimatorV2` to get ⟨Z on qubit 1⟩.

> ⚠️ These cells require:
> - a valid IBM Quantum account and
> - access to the given backend (e.g. `"ibm_sherbrooke"`).

In [9]:
import pennylane as qml
from qiskit_ibm_runtime import QiskitRuntimeService

try:
    # Connect to IBM Quantum / IBM Cloud account
    service = QiskitRuntimeService()

    # Choose a backend (explicit name used here; you can also call least_busy())
    backend = service.backend("ibm_sherbrooke")

    # Use the Pennylane-Qiskit "qiskit.remote" device to talk to Runtime backends
    num_qubits_supported = backend.num_qubits
    dev = qml.device("qiskit.remote", wires=num_qubits_supported, backend=backend)

    @qml.qnode(dev)
    def pl_hardware_circuit():
        """Simple 2-qubit circuit evaluated on IBM hardware via PennyLane."""
        qml.Hadamard(0)
        qml.CNOT(wires=[0, 1])
        return qml.expval(qml.PauliZ(1))

    print("[PennyLane] Hardware expval(Z_1):", pl_hardware_circuit())

except Exception as e:
    print("[PennyLane] Skipping hardware example due to:", e)


/Users/siddharthgolecha/Migration Testing/.venv/lib/python3.13/site-packages/pennylane_qiskit/qiskit_device.py:63: UserWarning: Expected an integer number of shots, but received shots=None. A default number of shots will be selected by the Qiskit backend. The analytic calculation of results is not supported on this device. All statistics obtained from this device are estimates based on samples.
  warnings.warn(


0.05795226699437294


In [12]:
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

try:
    # Connect to IBM Quantum Runtime
    service = QiskitRuntimeService()
    backend = service.backend("ibm_sherbrooke")

    # Logical circuit: note there are NO measurements here, since EstimatorV2
    # expects circuits without classical measurements.
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)

    # Observable: Z on qubit 1 (same convention as before)
    obs = SparsePauliOp.from_list([("ZI", 1.0)])

    # Compile logical circuit to backend's native gate set & layout
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    isa_circuit = pm.run(qc)

    # Update observable according to new layout (physical qubits vs logical)
    isa_observable = obs.apply_layout(isa_circuit.layout)

    # Runtime EstimatorV2 in 'mode=backend' executes on the target backend
    estimator = EstimatorV2(mode=backend)
    job = estimator.run([(isa_circuit, isa_observable)])
    result = job.result()[0]

    print("[Qiskit]    Hardware expval(Z_1):", result.data.evs)

except Exception as e:
    print("[Qiskit]    Skipping hardware example due to:", e)


0.006021451420686195


## 5. Variational Circuit + Gradient

**What we’re doing:**
- Define a 2-qubit variational circuit with parameter `θ`.
- Measure ⟨Z on qubit 1⟩ as the cost.
- Optimize `θ` using gradient-based methods.

### Conceptual mapping

| Concept                | PennyLane                                   | Qiskit (2.x)                                               |
|------------------------|---------------------------------------------|------------------------------------------------------------|
| Circuit container      | `@qml.qnode` + `default.qubit` device      | `QuantumCircuit`                                           |
| Observable             | `qml.expval(qml.PauliZ(1))`                | `SparsePauliOp("ZI")` (Z on qubit 1)                      |
| Execution primitive    | implicit in QNode                           | `EstimatorV2` (here from `qiskit-aer`)                     |
| Optimizer              | `GradientDescentOptimizer`                  | `GradientDescent` from `qiskit_algorithms.optimizers`      |
| Gradients              | analytic (parameter-shift)                  | finite differences (by default in `GradientDescent.minimize`) |


In [47]:
import pennylane as qml
from pennylane import numpy as pnp

# PennyLane device: statevector-based simulator
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev, interface="autograd")
def circuit(params):
    """PennyLane variational circuit:
    - Apply RY(theta) on qubit 0
    - Apply CNOT(0 -> 1)
    - Return expectation value of Z on qubit 1
    """
    qml.RY(params[0], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

# PennyLane built-in gradient-descent optimizer (uses analytic gradients where possible)
opt = qml.GradientDescentOptimizer(stepsize=0.1)

# Use PennyLane's numpy to ensure "trainable" parameters
params = pnp.array([0.5], requires_grad=True)

for i in range(20):
    params = opt.step(circuit, params)
    if i % 5 == 0:
        print(f"[PennyLane] Step {i}, cost = {circuit(params):.6f}")

[PennyLane] Step 0, cost = 0.853598
[PennyLane] Step 5, cost = 0.657084
[PennyLane] Step 10, cost = 0.291105
[PennyLane] Step 15, cost = -0.196028


In [48]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import EstimatorV2 as AerEstimator  # EstimatorV2 implementation (simulator)
from qiskit_algorithms.optimizers import GradientDescent

import numpy as np

# Define a parameterized 2-qubit circuit
qc = QuantumCircuit(2)
theta = Parameter('theta')
qc.ry(theta, 0)
qc.cx(0, 1)

# Observable: Z on qubit 1.
# In Qiskit Pauli strings, the RIGHT-most character is qubit 0.
# For 2 qubits, "ZI" = Z on qubit 1, I on qubit 0.
obs = SparsePauliOp.from_list([("ZI", 1.0)])

# EstimatorV2 from Aer: takes (circuit, observable, parameter_values)
est = AerEstimator()

def cost(x: np.ndarray) -> float:
    """Compute expectation value <Z_1> for given parameters x = [theta]."""
    t = float(x[0])
    # EstimatorV2.run takes a list of "pubs": (circuit, observables, parameter_values)
    pubs = [(qc, obs, [[t]])]

    job = est.run(pubs)
    pub_result = job.result()[0]        # first (and only) pub
    ev = np.asarray(pub_result.data.evs).item()  # scalar exp. value

    return float(ev)

# Set up GradientDescent optimizer (finite-difference gradient by default)
initial_point = np.array([0.5], dtype=float)

iter_counter = {"i": 0}
def callback(nfev, x, fx, grad_norm):
    """Log every 5th optimization step."""
    i = iter_counter["i"]
    if i % 5 == 0:
        print(f"[Qiskit]    Step {i}, cost = {fx:.6f}")
    iter_counter["i"] += 1

opt = GradientDescent(
    maxiter=20,
    learning_rate=0.1,
    callback=callback,
)

result = opt.minimize(fun=cost, x0=initial_point)

[Qiskit]    Step 0, cost = 0.853370
[Qiskit]    Step 5, cost = 0.654871
[Qiskit]    Step 10, cost = 0.286105
[Qiskit]    Step 15, cost = -0.201617
